In [4]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import finta as TA
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [5]:
input_path = '../data/raw/dados_historicos_ibovespa_2008-2025.csv'

df = pd.read_csv(input_path, thousands='.', decimal=',', parse_dates=['Data'], date_format='%d.%m.%Y', index_col='Data')
df = df.rename_axis('ds').sort_index()
df.tail()

,Último,Abertura,Máxima,Mínima,Vol.,Var%
ds,,,,,,
2025-06-12,137800,137127,137931,136175,"7,12B","0,49%"
2025-06-13,137213,137800,137800,136586,"8,63B","-0,43%"
2025-06-16,139256,137212,139988,137212,"7,62B","1,49%"
2025-06-17,138840,139256,139497,138293,"8,38B","-0,30%"
2025-06-18,138717,138844,139161,138443,"8,32B","-0,09%"


In [6]:
df.shape

(4315, 6)

In [7]:
# informações gerais do dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4315 entries, 2008-01-18 to 2025-06-18
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Último    4315 non-null   int64 
 1   Abertura  4315 non-null   int64 
 2   Máxima    4315 non-null   int64 
 3   Mínima    4315 non-null   int64 
 4   Vol.      4314 non-null   object
 5   Var%      4315 non-null   object
dtypes: int64(4), object(2)
memory usage: 236.0+ KB


In [8]:
# renomeando as colunas para os nomes padrões utilizados no mercado financeiro
colunas = {
  'Último': 'close',              # fechamento da negociação diária
  'Abertura': 'open',             # início da negociação diária
  'Máxima': 'high',               # valor máximo do dia
  'Mínima': 'low',                # valor mínimo do dia
  'Vol.': 'volume',               # volume de negociação diária
  'Var%': 'daily_return'          # variação percentual diária
}

df.rename(columns=colunas, inplace=True)

In [9]:
# Data mínima, máxima e total de anos do DF levando em conta os anos bissextos
print(f"Os dados vão de {df.index.min().date()} até {df.index.max().date()}, o que dá aproximadamente {(df.index.max() - df.index.min()).days / 365.25:.0f} anos")

Os dados vão de 2008-01-18 até 2025-06-18, o que dá aproximadamente 17 anos


In [10]:
# conferindo se há valores duplicados
df.duplicated().sum()

0

In [11]:
# conferindo se há valores nulos
df.isnull().sum()

close           0
open            0
high            0
low             0
volume          1
daily_return    0
dtype: int64

In [12]:
# conferindo se há valores nulos
df.isnull().sum().sort_values(ascending=False)

volume          1
close           0
open            0
high            0
low             0
daily_return    0
dtype: int64

In [13]:
def converter_volume(vol: str | float) -> float:
    """
    Converte uma string de volume com sufixos (K, M, B) para um número float.
    
    Parâmetro:
        vol (string | float): o valor a ser convertido (ex: '8,3M'). Pode ser uma string ou um np.nan (que é float).
        
    Retorna:
        float: o valor convertido ou np.nan caso não haja um valor.
    """
    if not isinstance(vol, str):
        return vol

    multiplicadores = {'K': 1e3, 'M': 1e6, 'B': 1e9}
    vol = vol.upper().replace(',', '.').strip()
    sufixo = vol[-1]

    if sufixo in multiplicadores:
        return float(vol[:-1]) * multiplicadores[sufixo]
    else:
        return float(vol)

df['volume'] = df['volume'].apply(converter_volume)

In [14]:
# substituir o volume nulo pela média do volume anterior e posterior daquela data
df['volume'] = df['volume'].interpolate()

In [15]:
# ajustando a coluna variação percentual diária, que contém o pct_change() do fechamento
df['daily_return'] = df['daily_return'].str.replace('%', '').str.replace(',', '.')
df['daily_return'] = round(df['daily_return'].astype(float) / 100, 4)
df.head()

,close,open,high,low,volume,daily_return
ds,,,,,,
2008-01-18,57506,57039,58291,56241,5810000.0,0.0082
2008-01-21,53709,57503,57503,53487,3570000.0,-0.0660
2008-01-22,56097,53705,56541,53610,3650000.0,0.0445
2008-01-23,54235,56098,56098,53011,3720000.0,-0.0332
2008-01-24,57463,54242,57675,54242,3800000.0,0.0595


In [16]:
# conferindo formato dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4315 entries, 2008-01-18 to 2025-06-18
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   close         4315 non-null   int64  
 1   open          4315 non-null   int64  
 2   high          4315 non-null   int64  
 3   low           4315 non-null   int64  
 4   volume        4315 non-null   float64
 5   daily_return  4315 non-null   float64
dtypes: float64(2), int64(4)
memory usage: 236.0 KB


In [17]:
df_prep = pd.DataFrame()

In [18]:
# Função de co-variáveis para o DataFrame dos modelos
def cria_variaveis_proximo_dia(df_input, df):
    # modulos
    import numpy as np
    from finta import TA

    # Diferença de fechamento e variações desconsiderando 0.5%
    # Variáveis Returns
    df_input['close'] = df['close'].copy()

    delta = df_input['close'].diff().copy() # Tira a diferença de um dia para o outro (d1 - d2)
    threshold = 0.005 # desconsidera variações menos que 0.5%
    df_input['target'] = np.where(
        delta > threshold, 1, np.where(delta < -threshold, 0, np.nan)
    )

    df_input['delta'] = delta.shift(1)
    df_input['return'] = df_input['close'].pct_change().shift(1)
    df_input['return_1d'] = df_input['close'].pct_change(1).shift()
    df_input['return_2d'] = df_input['close'].pct_change(2).shift()
    df_input['return_3d'] = df_input['close'].pct_change(3).shift()
    df_input['return_5d'] = df_input['close'].pct_change(5).shift()
    
    # Aceleração de momentum
    df_input['momentum_aceel'] = df_input['return_1d'] - df_input['return_2d']
    df_input['momentum_consistency'] = (
        (df_input['return_1d'] > 0) &
        (df_input['return_2d'] > 0) &
        (df_input['return_3d'] > 0)
    ).astype(int)
    
    # Volatilidade recente vs histórica
    df_input['vol_3d'] = df_input['return_1d'].rolling(3).std()
    df_input['vol_10d'] = df_input['return_1d'].rolling(10).std()
    df_input['vol_regime'] = df_input['vol_3d'] / df_input['vol_10d']

    # open, high, low
    df_input['open'] = df['open'].copy()
    df_input['high'] = df['high'].shift(1).copy()
    df_input['low'] = df['low'].shift(1).copy()
    
    # RSI
    df_input['rsi_9'] = TA.RSI(df_input, period=9).shift(1)
    df_input['rsi_14'] = TA.RSI(df_input, period=14).shift(1)
    df_input['rsi_divergence'] = df_input['rsi_14'] - df_input['rsi_9']
    
    # ADX
    df_input['adx'] = TA.ADX(df_input).shift(1)

    # Bollinger Bands
    rolling_mean = df_input['close'].rolling(20).mean().shift(1)
    rolling_std = df_input['close'].rolling(20).std().shift(1)
    
    df_input['bb_upper'] = rolling_mean + 2 * rolling_std
    df_input['bb_lower'] = rolling_mean - 2 * rolling_std
    df_input['bb_position'] = (df_input['close'] - df_input['bb_lower']) / (df_input['bb_upper'] - df_input['bb_lower'])
    
    # Z-Score
    df_input['z_score'] = (df_input['close'] - rolling_mean) / rolling_std

    # MACD rápido
    try:
        # Tenta passar o DataFrame inteiro primeiro
        macd_df = TA.MACD(df_input, column='close').shift(1)
        df_input['macd'] = macd_df['MACD']
        df_input['macd_signal'] = macd_df['SIGNAL']
        df_input['macd_hist'] = df_input['macd'] - df_input['macd_signal']
    except:
        # Retorno: calculo manual do MACD
        ema_12 = df_input['close'].ewm(span=12).mean().shift(1)
        ema_26 = df_input['close'].ewm(span=26).mean().shift(1)
        macd = ema_12 - ema_26
        macd_signal = macd.ewm(span=9).mean()
        
        df_input['macd'] = macd
        df_input['macd_signal'] = macd_signal
        df_input['macd_hist'] = macd - macd_signal
    # Stochastic rápido
    low_min = df_input['low'].copy().rolling(window=5).min()
    high_max = df_input['high'].copy().rolling(window=5).max()
    df_input['stoch_k'] = 100 * (df_input['close'].shift(1) - low_min) / (high_max - low_min)
    df_input['stoch_d'] = df_input['stoch_k'].rolling(window=3).mean()
    # Colunas criada algumas com defasagem de uma dia
    df_input['volatilidade'] = df_input['high'].copy() - df_input['low'].copy()
    df_input['volatilidade_relativa'] = df_input['high'].copy() / df_input['low'].copy()
    # MÉDIAS MÓVEIS E TENDÊNCIA (Peso: 10%)
    df_input['ema_5'] = df_input['close'].ewm(span=5).mean()
    df_input['ema_10'] = df_input['close'].ewm(span=10).mean()
    df_input['ema_cross'] = (df_input['ema_5'] > df_input['ema_10']).astype(int)

    return df_input

In [19]:
cria_variaveis_proximo_dia(df_prep,df)

df_prep = df_prep.dropna(axis=0)
df_prep

,close,target,delta,return,return_1d,return_2d,return_3d,return_5d,momentum_aceel,momentum_consistency,...,macd,macd_signal,macd_hist,stoch_k,stoch_d,volatilidade,volatilidade_relativa,ema_5,ema_10,ema_cross
ds,,,,,,,,,,,,,,,,,,,,,
2008-02-20,63747,1.0,-504.0,-0.008025,-0.008025,0.016729,0.007732,0.007961,-0.024754,0,...,789.325757,625.951656,163.374101,54.994779,63.825819,1410.0,1.022676,62568.895133,61768.503627,1
2008-02-21,63792,1.0,1450.0,0.023276,0.023276,0.015063,0.040394,0.018469,0.008212,1,...,890.799938,679.414419,211.385519,98.922625,79.171899,2146.0,1.034818,62976.651255,62140.917338,1
2008-02-22,64609,1.0,45.0,0.000706,0.000706,0.023998,0.015780,0.031916,-0.023292,1,...,961.766554,736.304622,225.461932,78.564129,77.493844,1011.0,1.015891,63520.815991,62594.145679,1
2008-02-25,65001,1.0,817.0,0.012807,0.012807,0.013522,0.037113,0.054462,-0.000715,1,...,1060.907949,801.610787,259.297163,99.341120,92.275958,1249.0,1.019713,64014.239972,63035.328401,1
2008-02-26,65183,1.0,392.0,0.006067,0.006067,0.018952,0.019672,0.035031,-0.012885,1,...,1152.335537,872.088559,280.246978,99.644865,92.516705,953.0,1.014877,64403.842077,63428.418728,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-12,137800,1.0,692.0,0.005072,0.005072,0.010531,0.007538,0.000920,-0.005459,1,...,397.082319,835.435830,-438.353511,88.188746,57.164526,1903.0,1.014031,137027.059649,137043.950398,0
2025-06-13,137213,0.0,672.0,0.004901,0.004901,0.009997,0.015483,0.011480,-0.005097,1,...,435.952197,755.539103,-319.586907,96.563484,78.093981,1756.0,1.012895,137089.039766,137074.686689,1
2025-06-16,139256,1.0,-587.0,-0.004260,-0.004260,0.000620,0.005695,0.008163,-0.004880,0,...,414.611478,687.353578,-272.742101,81.164743,88.638991,1214.0,1.008888,137811.359844,137471.289109,1


In [20]:
# Função lags de Series
def make_lags(series: pd.Series, n_lags):
    return series.shift(n_lags)

# Função cria colunas defasadas
def make_n_lags(df, n_lags, column, step):
    for i in range(1, n_lags + 1, step):
        df[f"{column}_lag{i}"] = df[column].shift(i)
    return df

In [21]:
n_lags = 7

# Chama função cria colunas com lags
df_prep = make_n_lags(df_prep, n_lags, "target", 1)
df_prep = make_n_lags(df_prep, n_lags, "close", 2)

In [22]:
df_prep

,close,target,delta,return,return_1d,return_2d,return_3d,return_5d,momentum_aceel,momentum_consistency,...,target_lag2,target_lag3,target_lag4,target_lag5,target_lag6,target_lag7,close_lag1,close_lag3,close_lag5,close_lag7
ds,,,,,,,,,,,,,,,,,,,,,
2008-02-20,63747,1.0,-504.0,-0.008025,-0.008025,0.016729,0.007732,0.007961,-0.024754,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-02-21,63792,1.0,1450.0,0.023276,0.023276,0.015063,0.040394,0.018469,0.008212,1,...,NaN,NaN,NaN,NaN,NaN,NaN,63747.0,NaN,NaN,NaN
2008-02-22,64609,1.0,45.0,0.000706,0.000706,0.023998,0.015780,0.031916,-0.023292,1,...,1.0,NaN,NaN,NaN,NaN,NaN,63792.0,NaN,NaN,NaN
2008-02-25,65001,1.0,817.0,0.012807,0.012807,0.013522,0.037113,0.054462,-0.000715,1,...,1.0,1.0,NaN,NaN,NaN,NaN,64609.0,63747.0,NaN,NaN
2008-02-26,65183,1.0,392.0,0.006067,0.006067,0.018952,0.019672,0.035031,-0.012885,1,...,1.0,1.0,1.0,NaN,NaN,NaN,65001.0,63792.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-12,137800,1.0,692.0,0.005072,0.005072,0.010531,0.007538,0.000920,-0.005459,1,...,1.0,0.0,0.0,0.0,0.0,1.0,137128.0,135699.0,136236.0,137546.0
2025-06-13,137213,0.0,672.0,0.004901,0.004901,0.009997,0.015483,0.011480,-0.005097,1,...,1.0,1.0,0.0,0.0,0.0,0.0,137800.0,136436.0,136102.0,137002.0
2025-06-16,139256,1.0,-587.0,-0.004260,-0.004260,0.000620,0.005695,0.008163,-0.004880,0,...,1.0,1.0,1.0,0.0,0.0,0.0,137213.0,137128.0,135699.0,136236.0


In [23]:
df_model_data = df_prep.copy()
df_model_data['daily_return'] = df['daily_return'].copy()
df_model_data['daily_return']
df_model_data = df_model_data[df_model_data.index >= '2015-06-17'].dropna().copy()

In [24]:
# Exclui tabela Close que não vai ser usada
df_prep.drop('close', axis=1, inplace=True)
# Data Frame pronto para o modelo doprando linhas com valores na
df_model = df_prep.dropna().copy()

In [25]:
# Salva arquivo removendo o index
df_model.to_csv('../data/processed/arquivo-modelo.csv', index=False)

In [26]:
# Salva arquivo mantendo o index
df_model_data.to_csv('../data/processed/arquivo-exploratorio.csv', index=True)